In [12]:
# imports
import random
import numpy as np

In [13]:
REWARDS = {" ":0.0, "D":-1.0, "J":12.0, "S":0.0}
DIRECTIONS = [
    "HAUT",
    "DROITE",
    "BAS",
    "GAUCHE",
]


Nparties = 10000
Ncoups = 100
alpha = 0.81
gamma = 0.96

In [14]:
def init_random_space(lines, columns, nbDragon):
    space = []
    for l in range(lines):
        space.append([])
        for c in range(columns):
            space[l].append(' ')
    space[0][0] = 'S'
    space[lines-1][columns-1] = 'J'

    i = 0
    while i < nbDragon: 
        l = random.randint(0, lines-1)
        c = random.randint(0, columns-1)
        if space[l][c] == ' ':
            space[l][c] = 'D'
            i += 1
        else:
            i -= 1
    return space



def init_space():
    space = init_random_space(4, 4, 3)
    return space;


def isWin (space, position):
    (l,c) = position

    if(space[l][c]== 'J'):
        return True

def application_action(action, position, space):

    # on pénalise l'action
    reward = -1.0
    
    (l, c) = position

    nextPos = position
    if action == "HAUT":
        nextPos = (l-1,c)
    elif action == "DROITE":
        nextPos = (l,c+1)
    elif action == "BAS":
        nextPos = (l+1,c);
    elif action == "GAUCHE":
        nextPos = (l,c-1);

    # ne pas sortir de la grille
    if (nextPos[0] < len(space) and nextPos[1] < len(space) and nextPos[0] >=0 and nextPos[1] >=0 ):
        position = nextPos
        # get la case dans le space 
        case = space[position[0]][position[1]]
        if (case == 'D'):
            position = (0,0)

        reward += REWARDS[case]
    else:
        reward -= 1.0
        
    fin = isWin(space, position)
    if fin:
        position = (0,0)
    return position, reward, fin

In [15]:
space = init_space();
print(space)
#position ddu chevalier
player_pos = (0, 0)

# for i in range(Nparties):
#     print("------------------------------- tour ", i)
#     # on choisit une direction aléatoire
#     action = random.choice(DIRECTIONS)
#     print("action : ", action)
#     # on applique l'action
#     player_pos, reward, fin = application_action(action, player_pos, space)
#     print("position : ", player_pos)
#     print("reward : ", reward)
#     print("fin : ", fin)

#     # fin de partie
#     if fin:
#         print("fin de partie")
#         break

[['S', ' ', ' ', 'D'], [' ', ' ', ' ', 'D'], [' ', ' ', 'D', 'D'], [' ', ' ', ' ', 'J']]


# 2. Développement du Q-learning

In [16]:
def choose_action(state, epsilon, mat_q):
    if random.random() < epsilon:
        # on choisit une action aléatoire
        action = random.choice(DIRECTIONS)
    else:
        # on choisit l'action qui maximise la matrice Q
        #print("state", state, "mat_q[state] : ", mat_q[state], "action : ", DIRECTIONS[np.argmax(mat_q[state])])
        action = DIRECTIONS[np.argmax(mat_q[state])]
    return action

In [17]:
def oneStep(mat_q, state, epsilon):
    # on choisit une action
    action = choose_action(state, epsilon, mat_q)
    # on applique l'action
    new_state, reward, fin = application_action(action, state, space)
    # on met à jour la matrice Q
    #mat_q[state][DIRECTIONS.index(action)] += alpha * (reward + gamma * (mat_q[new_state][DIRECTIONS.index(choose_action(new_state, epsilon, mat_q))]) - mat_q[state][DIRECTIONS.index(action)])
    mat_q[state][DIRECTIONS.index(action)] += alpha * (reward + gamma * np.max(mat_q[new_state]) - mat_q[state][DIRECTIONS.index(action)])
    return mat_q, new_state, fin

In [18]:
# create a mat_q
mat_q = np.zeros((len(space), len(space), len(DIRECTIONS)))

totalSteps = 0

# on applique l'algorithme
for iterationPartie in range(Nparties):
    state = (0,0)
    # calcul de epsilon
    epsilon = Nparties/(Nparties+iterationPartie)
    #print("epsilon : ", epsilon)

    for iterationCoups in range(Ncoups):
        mat_q, state, fin = oneStep(mat_q, state, epsilon)
        if fin:
            #print("fin de partie en ", iterationCoups, " coups, partie ", iterationPartie)
            totalSteps += iterationCoups
            #print(mat_q)
            break

print("nombre de coups total : ", totalSteps)
print("moyenne de coups par partie : ", totalSteps/Nparties)

#print(mat_q)


nombre de coups total :  255447
moyenne de coups par partie :  25.5447


In [19]:

for iterationCoups in range(Ncoups):
    mat_q, state, fin = oneStep(mat_q, state, 0)
    if fin:
        print("fin de partie en ", iterationCoups, " coups")
        break


fin de partie en  5  coups
